# <font color=blue>Affine gaps global alignent</font>

Задача состоит в поиске расстояния Левенштейна для двух заданных строк. На вход подаются две сравниваемые строки, матрица весов (на выбор default.txt или PAM250.txt), вес первого gap-а и вес всех последующих. Код 
```
    1) читает матрицу весов и выводит её
    2) Расчитывает матрицу расстояний между префиксами и выводит её
    3) Восстанавливает и выводит выровненные строки
```

In [22]:
with open('default.txt', 'w') as f:
    f.write('C\t10\n')
    f.write('A\t-2\t10\n')
    f.write('R\t-2\t-2\t10\n')
    f.write('T\t-2\t-2\t-2\t10\n')
    f.write('S\t-2\t-2\t-2\t-2\t10\n')
    f.write(' \tC\tA\tR\tT\tS\n')
    
    
#    f.write('C\t0\n')
#    f.write('S\t-1\t0\n')
#    f.write('T\t-1\t-1\t0\n')
#    f.write('P\t-1\t-1\t-1\t0\n')
#    f.write('A\t-1\t-1\t-1\t-1\t0\n')
#    f.write('G\t-1\t-1\t-1\t-1\t-1\t0\n')
#    f.write('N\t-1\t-1\t-1\t-1\t-1\t-1\t0\n')
#    f.write(' \tC\tS\tT\tP\tA\tG\tN\n')

In [62]:
# Функции

def PrintMatrix(Seq1, Seq2, matrix):                                # Функция печати матрицы расстояний между префиксами
    OutputString = ' \t-\t'                                         # отдельно печатаем первую строчку, содержащую seq1
    for j in range(len(Seq1)):   
        OutputString += Seq1[j] + '\t'
    print(OutputString)
    
    OutputString = '-\t'                                            # печатаем оставшиеся 
    for j in range(len(Seq2)+1):                                    # Итерация по строчкам вниз
        for i in range(len(Seq1)+1):                                # Итерация по столбцам вправо
            OutputString += str(matrix[i][j]) + '\t'
        print(OutputString)
        if j < len(Seq2): OutputString = Seq2[j] + '\t'
    return None

def GetMatrix(filename, Dictionary):                                 # С помощью матрицы, записанной в файле,
    with open(filename, 'r') as f:                                   # заполняем словарь, с которым удобно работать
        text = f.read()
    print('весовая матрица')
    print(text)
    l1 = text.split('\n')
    l2 = []
    for i in range(len(l1)-1):
        l2.append(l1[i].split('\t'))
   
    for i in range (len(l2)-1):
        for j in range (1, len(l2[i])):
            Dictionary[str(l2[i][0]) + str(l2[len(l2)-1][j])] = l2[i][j]
            Dictionary[str(l2[len(l2)-1][j]) + str(l2[i][0])] = l2[i][j]
    return None



# Начало

# Принимаем на вход сравниваемые строчки
Sequence1 = 'PRTEINS'      #str(input())
Sequence2 = 'PRTWPSEIN'       #str(input())

# Инициализируем словарь и записываем в него матрицу весов
D = {}
GetMatrix('BLOSUM.txt', D)

# Задаем вес пропуска
Dfirstgap = -11
Dothergaps = -1

# Элементы матриц, которые не должны существовать, будем для удобства инициализировать числом
MinusInfinity = -100



# Инициализация матриц

# Инициализируем матрицу расстояний между префиксами, кончающимися на match или mismatch
M = [[0]*(len(Sequence2)+ 1) for _ in range (len(Sequence1)+ 1)]  

# заполняем дефолтные позиции этой матрицы
M[0][0] = 0
for i in range(1, len(Sequence1)+1):
    M[i][0] = MinusInfinity
for j in range(1, len(Sequence2)+1):
    M[0][j] = MinusInfinity

# Инициализируем матрицу расстояний между префиксами, у которых outputsequence1 кончается gap-ом
X = [[0]*(len(Sequence2)+ 1) for _ in range (len(Sequence1)+ 1)]

# заполняем дефолтные позиции этой матрицы
X[0][0] = 0
X[0][1] = Dfirstgap
for i in range(1, len(Sequence1)+1):                            # Не существует такого выравнивания префикса строки Sequence1
    X[i][0] = MinusInfinity                                     # на пустую строку, чтобы outputsequence1 кончался gap-ом
for j in range(2, len(Sequence2)+1):
    X[0][j] = X[0][j-1] + Dothergaps   
    
# Инициализируем матрицу расстояний между префиксами, у которых outputsequence2 кончается gap-ом
Y= [[0]*(len(Sequence2)+ 1) for _ in range (len(Sequence1)+ 1)]

# заполняем дефолтные позиции этой матрицы
Y[0][0] = 0
Y[1][0] = Dfirstgap
for i in range(2, len(Sequence1)+1):                            
    Y[i][0] = Y[i-1][0] + Dothergaps                        
for j in range(1, len(Sequence2)+1):                            # Не существует такого выравнивания префикса строки Sequence2
    Y[0][j] = MinusInfinity                                     # на пустую строку, чтобы outputsequence2 кончался gap-ом

    
    
# Получаем матрицы расстояний

# последовательно считаем оставшиеся элементы матриц
for i in range(1, len(Sequence1) + 1):                                # Итерация по столбцам вправо
    for j in range(1, len(Sequence2) + 1):                            # Итерация по строчкам вниз
        X[i][j] = max((Dfirstgap + M[i][j-1]), (Dothergaps + X[i][j-1]), (Dfirstgap + Y[i][j-1]))
        Y[i][j] = max((Dfirstgap + M[i-1][j]), (Dfirstgap + X[i-1][j]), (Dothergaps + Y[i-1][j]))
        M[i][j] = max((float(D[Sequence1[i-1] + Sequence2[j-1]]) + M[i-1][j-1]), \
                       (Y[i][j]), \
                       (X[i][j]))   # строка начинается с нулевого элемента
    
# печатаем получившуюся матрицы
print('\n\n\nM')
PrintMatrix(Sequence1, Sequence2, M)
print('\n\n\nX')
PrintMatrix(Sequence1, Sequence2, X)
print('\n\n\nY')
PrintMatrix(Sequence1, Sequence2, Y)



# восстанавливаем выровненные последолвательности

# инициализируем выводимые последовательности
outputsequence1 = ''
outputsequence2 = ''

print('\n\n\n')

i, j = len(Sequence1), len(Sequence2)
print(i, ' ', j, '\n')
while i > 0 or j > 0:    
    
    if  j > 0 and M[i][j] == X[i][j]:
        outputsequence1 = '-' + outputsequence1
        outputsequence2 =  Sequence2[j - 1] + outputsequence2
        i, j = i, j - 1
    elif i > 0 and M[i][j] == Y[i][j]:
        outputsequence1 = Sequence1[i-1] + outputsequence1
        outputsequence2 = '-' + outputsequence2
        i, j = i - 1, j
    elif i > 0 and j > 0 and M[i][j] == float(D[Sequence1[i-1] + Sequence2[j-1]]) + M[i-1][j-1]:
        outputsequence1 = Sequence1[i - 1]  + outputsequence1
        outputsequence2 = Sequence2[j - 1] + outputsequence2
        i, j = i - 1, j - 1
    print(i, ' ', j)


    Value1, Value2, Value3 = 0, 0, 0

    
print('\n\n\nВыровненные слова')
print(outputsequence1)
print(outputsequence2)

весовая матрица
A	4
C	0	9
D	-2	-3	6
E	-1	-4	2	5
F	-2	-2	-3	-3	6
G	0	-3	-1	-2	-3	6
H	-2	-3	-1	0	-1	-2	8
I	-1	-1	-3	-3	0	-4	-3	4
K	-1	-3	-1	1	-3	-2	-1	-3	5
L	-1	-1	-4	-3	0	-4	-3	2	-2	4
M	-1	-1	-3	-2	0	-3	-2	1	-1	2	5
N	-2	-3	1	0	-3	0	1	-3	0	-3	-2	6
P	-1	-3	-1	-1	-4	-2	-2	-3	-1	-3	-2	-2	7
Q	-1	-3	0	2	-3	-2	0	-3	1	-2	0	0	-1	5
R	-1	-3	-2	0	-3	-2	0	-3	2	-2	-1	0	-2	1	5
S	1	-1	0	0	-2	0	-1	-2	0	-2	-1	1	-1	0	-1	4
T	0	-1	-1	-1	-2	-2	-2	-1	-1	-1	-1	0	-1	-1	-1	1	5
V	0	-1	-3	-2	-1	-3	-3	3	-2 	1	1	-3	-2	-2	-3	-2	0	4
W	-3	-2	-4	-3	1	-2	-2	-3	-3	-2	-1	-4	-4	-2	-3	-3	-2	-3	11
Y	-2	-2	-3	-2	3	-3	2	-1	-2	-1	-1	-2	-3	-1	-2	-2	-2	-1	2	7
	A	C	D	E	F	G	H	I	K	L	M	N	P	Q	R	S	T	V	W	Y




M
 	-	P	R	T	E	I	N	S	
-	0	-100	-100	-100	-100	-100	-100	-100	
P	-100	7.0	-4.0	-5.0	-6.0	-7.0	-8.0	-9.0	
R	-100	-4.0	12.0	1.0	0.0	-1.0	-2.0	-3.0	
T	-100	-5.0	1.0	17.0	6.0	5.0	4.0	3.0	
W	-100	-6.0	0.0	6.0	14.0	3.0	2.0	1.0	
P	-100	-7.0	-1.0	5.0	5.0	11.0	1.0	1.0	
S	-100	-8.0	-2.0	4.0	5.0	3.0	12.0	5.0	
E	-100	-9.0	-3.0	3.0	9.0	2.0	3.0	12